In [1]:
# Script to adapt on html pagges

In [2]:
cd .

/var/www/kerlandrier_mono/api/script/scraping


In [ ]:
import sys
import os
import git
import pandas as pd
from dateparser import parse as dparse

# Ajoute le dossier "ressources" au sys.path
git_root = git.Repo(search_parent_directories=True).working_tree_dir
sys.path.insert(0,   os.path.abspath(  os.path.join(  git_root,'api' ) ) )

import script.libs.utils as utils
import script.libs.scraping_utils as scraping_utils
import script.libs.HttpRequests as HttpRequests
from script.configuration import config, oa
from script.libs.getOaLocation import get_or_create_oa_location

from slugify import slugify
import os
import requests
# import requests_cache
from bs4 import BeautifulSoup
import  dateparser, pytz
from urllib.parse import urlparse

In [5]:
baseURL= "https://www.festivaldesfiletsbleus.bzh/"
allEventsPage= f"{baseURL}/programme_filets_bleus_2025.html"
# locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
# DEFAULT_TIME_START = "20:00:00"
# DEFAULT_TIME_END = "22:00:00"
getTimeout= scraping_utils.getTimeout

In [6]:
html_doc = requests.get(allEventsPage, timeout=getTimeout).content.decode('ISO-8859-1',errors='replace')
parsed_html = BeautifulSoup(html_doc ,'html.parser')

In [7]:
dateSelector= "div > h6"
extractedDates= extractedConcerts = parsed_html.select(dateSelector)
dates = [f"{d.text} 2025" for d in extractedDates if d.text.strip() != ""]
print("Dates:", dates)

Dates: ['MERCREDI 13 AOÃ\x9bT 2025', 'JEUDI 14 AOÃ\x9bT 2025', 'VENDREDI 15 AOÃ\x9bT 2025', 'SAMEDI 16 AOÃ\x9bT 2025', 'DIMANCHE 17 AOÃ\x9bT 2025']


In [11]:
# Selector pour le contenu de chaque Event
import pprint

allEventsSelector= "span.PROGTXT p" #encarts où cse trouve la liste des événements
for e in parsed_html.find_all('br'):
    e.extract()
extractedConcerts = parsed_html.select(allEventsSelector)
concerts=[]

for d,part in enumerate(extractedConcerts):
    splitted =part.get_text().strip().split('\n')
    i=0
    for c in splitted:
        if c.find(":") <0 and i > 0: #merging lines from the same event
            concerts[-1] = f"{(splitted[i-1]+ " " + splitted[i]).replace(":","__")} __ {dates[d]} __ {baseURL}{part.select("a")[i-1]["href"].strip()}"
            c=c.replace(":","__")
            continue
        c=c.replace(":","__")
        concerts.append( f"{c.strip()} __ {dates[d]} __ {baseURL}{part.select("a")[i]["href"].strip()}" )
        i+=1

# print(concerts)
concerts = [c.split("__") for c in concerts]
# concerts = [e.strip() for c in concerts for e in c]
print(concerts)
pprint.pprint(concerts[2:])



[['19h00-20h30 ', ' LASS ', ' MERCREDI 13 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Lass'], ['20h45-22h15 ', ' SHARON SHANNON ', ' MERCREDI 13 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Sharon'], ['22h30-00h00 ', ' SONERIEN DU ', ' MERCREDI 13 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Sonerien'], ['19h00-20h30 ', ' CERCLE CELTIQUE AR ROUEDOÃ\x99 GLAS ', ' JEUDI 14 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Cercle'], ['20h45-22h15 ', ' SOLDAT LOUIS ', ' JEUDI 14 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Soldat'], ['22h30-00h00 ', " O'TRIDAL ", ' JEUDI 14 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#OTridal'], ['19h00-20h30 ', ' BATTERIE FANFARE ECHO DU MONT BLANC ', ' VENDREDI 15 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Fanfare'], ['20h45-22h15 ', ' 

In [ ]:
access_token = oa.getToken()

timezone_paris = pytz.timezone('Europe/Paris')

locationUid= get_or_create_oa_location("Quai Carnot, Concarneau",
                                        access_token,
                                        oa.public_key,
                                        f"{config.OA_API_URL}/locations")


INFO:script.libs.getOaLocation:- searching location for : 'Quai Carnot, Concarneau'
INFO:script.libs.getOaLocation: (optimized name for better matching:  'Quai Carnot Concarneau')
INFO:script.libs.getOaLocation:- 🎯 Location found in OA: ('Quai Carnot Quai Carnot, 29900 Concarneau, France', 100, 37681236)


In [45]:
c=concerts[5]
date=f"{c[0].split("-")[1].strip().replace("00h00","23h50")} {c[2].strip().replace("AOÃ\x9bT", "aout")}"
print(date)
datetz=dateparser.parse(date).astimezone(timezone_paris).isoformat()
print(datetz)


23h50 JEUDI 14 aout 2025
2025-08-14T23:50:00+02:00


In [ ]:
concertsD= []
for index,c in enumerate(concerts):
    datedebut=dparse(f"{c[0].split("-")[0].strip().replace("00h00","23h50")} {c[2].strip().replace("AOÃ\x9bT", "aout")}").astimezone(timezone_paris).isoformat()
    datefin=dparse(f"{c[0].split("-")[1].strip().replace("00h00","23h50")} {c[2].strip().replace("AOÃ\x9bT", "aout")}").astimezone(timezone_paris).isoformat()
    concertsD.append({
        "uid-externe": "manual_scrap-filets-bleus-" + slugify(f"{c[1].strip()}"),
        "title": { "fr": f"{c[1].strip().capitalize()}"},
        "description": { "fr": f"Concert de {c[1].strip().capitalize()} - Scène Armor"},
        "longDescription": f"Concert de {c[1].strip().capitalize()} - Scène Armor",
        "timings": [
            {
                "begin": datedebut,
                "end": datefin
            }
        ],
        "lien": c[3].strip(),
        "locationUid": locationUid,
        "keywords": ['filets bleus', 'concert']
    }
    )

pprint.pprint(concertsD)

[{'description': {'fr': 'Concert de Lass - Scène Armor'},
  'keywords': ['filets bleus', 'concert'],
  'lien': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Lass',
  'locationUid': 37681236,
  'longDescription': 'Concert de Lass - Scène Armor',
  'timings': [{'begin': '2025-08-13T19:00:00+02:00',
               'end': '2025-08-13T20:30:00+02:00'}],
  'title': {'fr': 'LASS'},
  'uid-externe': 'manual_scrap-filets-bleus-lass'},
 {'description': {'fr': 'Concert de Sharon shannon - Scène Armor'},
  'keywords': ['filets bleus', 'concert'],
  'lien': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Sharon',
  'locationUid': 37681236,
  'longDescription': 'Concert de Sharon shannon - Scène Armor',
  'timings': [{'begin': '2025-08-13T20:45:00+02:00',
               'end': '2025-08-13T22:15:00+02:00'}],
  'title': {'fr': 'SHARON SHANNON'},
  'uid-externe': 'manual_scrap-filets-bleus-sharon-shannon'},
 {'description': {'fr': 'Concert de Sonerien du - Scène Armor'},
  'keyw

In [47]:
for event in concertsD:
    print(event)
    print (event.get("title"))
    response = HttpRequests.create_event(access_token,
                                        event,
    )
    print(response)

{'uid-externe': 'manual_scrap-filets-bleus-lass', 'title': {'fr': 'LASS'}, 'description': {'fr': 'Concert de Lass - Scène Armor'}, 'longDescription': 'Concert de Lass - Scène Armor', 'timings': [{'begin': '2025-08-13T19:00:00+02:00', 'end': '2025-08-13T20:30:00+02:00'}], 'lien': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Lass', 'locationUid': 37681236, 'keywords': ['filets bleus', 'concert']}
{'fr': 'LASS'}


INFO:root:New event: "LASS"                         https://openagenda.com/kerlandrier/contribute/event/89977579


{'success': True, 'event': {'uid': 89977579, 'slug': 'lass-1401465', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:48:56.344Z', 'createdAt': '2025-08-04T11:48:56.344Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'ee74f5cdeb3e42d1b6ab7e352d5c519c', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'LASS'}, 'description': {'fr': 'Concert de Lass - Scène Armor'}, 'longDescription': {'fr': 'Concert de Lass - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-13T19:00:00.000+02:00', 'end': '2025-08-13T20:30:00.000+02:00'}], 'status': 1, 'links': [], 'location': {'uid': 37681236, 'setUid': None, 'slug': 'quai-carnot-29900-concarneau-fr

INFO:root:New event: "SHARON SHANNON"                         https://openagenda.com/kerlandrier/contribute/event/28249087


{'success': True, 'event': {'uid': 28249087, 'slug': 'sharon-shannon-6906819', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:48:57.000Z', 'createdAt': '2025-08-04T11:48:56.945Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': '2889b890e8734b2a9e8ab63bf2227e3f', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'SHARON SHANNON'}, 'description': {'fr': 'Concert de Sharon shannon - Scène Armor'}, 'longDescription': {'fr': 'Concert de Sharon shannon - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-13T20:45:00.000+02:00', 'end': '2025-08-13T22:15:00.000+02:00'}], 'status': 1, 'links': [], 'location': {'uid': 37681236, 'setUid': None, 

...

{'success': True, 'event': {'uid': 67609285, 'slug': 'batterie-fanfare-echo-du-mont-blanc-5079176', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:49:00.000Z', 'createdAt': '2025-08-04T11:48:59.526Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': '3c5543abe427436e9465172325c41977', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'BATTERIE FANFARE ECHO DU MONT BLANC'}, 'description': {'fr': 'Concert de Batterie fanfare echo du mont blanc - Scène Armor'}, 'longDescription': {'fr': 'Concert de Batterie fanfare echo du mont blanc - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-15T19:00:00.000+02:00', 'end': '2025-08-15T20:30:00.00

INFO:root:New event: "BOULEVARD DES AIRS"                         https://openagenda.com/kerlandrier/contribute/event/35888069


{'success': True, 'event': {'uid': 35888069, 'slug': 'boulevard-des-airs-7013859', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:49:00.142Z', 'createdAt': '2025-08-04T11:49:00.142Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': '5846f3e1a72b4b299f813203ce7d1882', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'BOULEVARD DES AIRS'}, 'description': {'fr': 'Concert de Boulevard des airs - Scène Armor'}, 'longDescription': {'fr': 'Concert de Boulevard des airs - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-15T20:45:00.000+02:00', 'end': '2025-08-15T22:15:00.000+02:00'}], 'status': 1, 'links': [], 'location': {'uid': 37681236, 

INFO:root:New event: "AMPOUAILH"                         https://openagenda.com/kerlandrier/contribute/event/60279731


{'success': True, 'event': {'uid': 60279731, 'slug': 'ampouailh-6042500', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:49:01.000Z', 'createdAt': '2025-08-04T11:49:00.542Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': '5ced686a03e643a2a6e74b870e408673', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'AMPOUAILH'}, 'description': {'fr': 'Concert de Ampouailh - Scène Armor'}, 'longDescription': {'fr': 'Concert de Ampouailh - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-15T22:30:00.000+02:00', 'end': '2025-08-15T23:50:00.000+02:00'}], 'status': 1, 'links': [], 'location': {'uid': 37681236, 'setUid': None, 'slug': 'quai-carnot